In [1]:
import torch
#from denoising_diffusion_pytorch import Unet, GaussianDiffusion
#from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D
from denoising_diffusion_pytorch import *
import numpy as np
import matplotlib.pyplot as plt
#from scipy.stats import skew,kurtosis

In [ ]:
pip install torch torchvision numpy matplotlib einops ema_pytorch accelerate pytorch_fid

## Load the data

In [2]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_bhat/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [3]:
data = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [4]:
# (40, 40, 99) (videonumber/trialnumber , channelnumber, datapoints)
data['sub01'].shape

(40, 40, 99)

In [5]:
# matrix 1
data['sub01'][0].shape

(40, 99)

In [6]:
matrix = np.zeros((1280,3,40,104))
matrix[:] = np.nan
counter = 0
check = np.zeros((104,40))
for sub in data.keys():
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        check = temp
        matrix[counter, :] = temp
        counter = counter + 1

In [7]:
matrix.shape

(1280, 3, 96, 40)

In [8]:
np.isnan(matrix).any()

False

In [9]:
matrix[0][0]

array([[0.29633097, 0.540126  , 0.55221434, ..., 0.34482342, 0.90617533,
        0.35146873],
       [0.5152931 , 0.55177657, 0.31579877, ..., 0.46382535, 0.94400325,
        0.27318548],
       [0.05009588, 0.        , 0.        , ..., 0.1260563 , 0.67083414,
        0.20812183],
       ...,
       [0.85168198, 0.88137392, 0.91328749, ..., 0.88934937, 0.60875284,
        0.75261257],
       [0.23165782, 0.22880939, 0.18914166, ..., 0.05785734, 0.53919537,
        0.1411673 ],
       [0.19900799, 0.1860128 , 0.14288434, ..., 0.0123726 , 0.38281514,
        0.04122104]])

In [10]:
tensor_matrix = torch.tensor(matrix).float()

In [11]:
tensor_matrix.shape

torch.Size([1280, 3, 96, 40])

In [12]:
image_size = (99,40)
image_size[0]

99

In [ ]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
)

diffusion = GaussianDiffusion(
    model,
    image_size = (40,104),
    timesteps = 1000,   # number of steps
    loss_type = 'l1'    # L1 or L2
)

training_images = tensor_matrix # images are normalized from 0 to 1
#training_images = torch.rand(8, 3, 96, 40) # images are normalized from 0 to 1
loss = diffusion(training_images)
loss.backward()
# after a lot of training

sampled_images = diffusion.sample(batch_size = 4)
sampled_images.shape # (4, 3, 128, 128)

HELLLO
